# Monitoring & Action Plan Proposal

**MOAZ**

---

## Imports 

In [1]:
import sys
sys.path.append("../src")

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

from data.loader import load_excel_sheets
from analysis.supply_demand import compute_subcategory_supply, compute_subcategory_demand, merge_supply_demand
from analysis.gap_score import compute_gap_score, normalize_gap_score, classify_gap_level, compute_demand_supply_ratio
from monitoring.engine import MonitoringSnapshot, GapChangeAnalyzer, compute_kpis
from monitoring.action_planner import ActionPlanner
from monitoring.visualizations import (
    create_monitoring_kpi_cards,
    create_gap_trend_chart,
    create_alert_table,
    create_change_heatmap,
    create_supply_demand_balance_chart
)
from data.exporter import export_plotly_chart, export_insights_to_text

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

print("Environment configured successfully")

Environment configured successfully


---

## Data Loading & Preparation

In [2]:
file_path = "../data/processed/Cleaned_data.xlsx"

product_lookup = pd.read_excel(file_path, sheet_name=' Product Lookup')
product_subcategory = pd.read_excel(file_path, sheet_name='Product Subcategory')
product_category = pd.read_excel(file_path, sheet_name='Product Category')
sales_data = pd.read_excel(file_path, sheet_name='Sales Data')

products_with_subcat = product_lookup.merge(product_subcategory, on='ProductSubcategoryKey', how='left')
products_full = products_with_subcat.merge(product_category, on='ProductCategoryKey', how='left')

sales_full = sales_data.merge(
    products_full[['ProductKey', 'CategoryName', 'SubcategoryName']],
    on='ProductKey',
    how='left'
)

print(f"Data loaded: {sales_full['CategoryName'].nunique()} categories, {sales_full['SubcategoryName'].nunique()} subcategories")

Data loaded: 3 categories, 17 subcategories


---

## Gap Score Calculation

In [3]:
supply_df = compute_subcategory_supply(sales_data, products_full)
demand_df = compute_subcategory_demand(sales_full)
supply_demand_df = merge_supply_demand(supply_df, demand_df)

gap_df = compute_gap_score(supply_demand_df, demand_col='TotalQuantitySold', supply_col='UniqueProducts')
gap_df = normalize_gap_score(gap_df, gap_col='GapScore')
gap_df = classify_gap_level(gap_df, gap_col='GapScore')
gap_df = compute_demand_supply_ratio(gap_df)

gap_summary = gap_df[['CategoryName', 'SubcategoryName', 'UniqueProducts', 
                       'TotalQuantitySold', 'GapScore', 'GapLevel']].copy()
gap_summary.columns = ['parent_category', 'category', 'supply', 'demand', 'gap_score', 'gap_status']
gap_summary = gap_summary.sort_values('gap_score', ascending=False).reset_index(drop=True)

print(f"\nGap analysis computed for {len(gap_summary)} subcategories")
display(gap_summary.head(10))


Gap analysis computed for 37 subcategories


,parent_category,category,supply,demand,gap_score,gap_status
0,Accessories,Bottles and Cages,3,15106,"3,776.75",Critical Gap
1,Accessories,Tires and Tubes,11,29772,"2,481.08",Critical Gap
2,Clothing,Caps,1,4151,"2,076.00",Critical Gap
3,Accessories,Fenders,1,3960,"1,980.50",Critical Gap
4,Accessories,Helmets,3,6034,"1,508.75",Critical Gap
5,Accessories,Cleaners,1,1706,853.50,Critical Gap
6,Clothing,Gloves,6,2644,377.86,Critical Gap
7,Accessories,Hydration Packs,1,695,348.00,Critical Gap
8,Clothing,Jerseys,8,3113,346.00,Critical Gap
9,Clothing,Socks,4,1063,212.80,Critical Gap


---

## Create Monitoring Snapshot

In [4]:
week_number = datetime.now().isocalendar()[1]

snapshot_engine = MonitoringSnapshot()
json_path, csv_path = snapshot_engine.save_snapshot(gap_summary, week_number=week_number)

print(f"Snapshot saved:")
print(f"  JSON: {json_path}")
print(f"  CSV: {csv_path}")

Snapshot saved:
  JSON: /home/hamza/All Data To Transfer/VIF/Materials/Fourth Year/Graduation Project/1st semester/Gap Analysis/v4.0/rently_2026/Gap_Analysis/notebooks/../src/results/monitoring_reports/snapshot_week_2_2026-01-09.json
  CSV: /home/hamza/All Data To Transfer/VIF/Materials/Fourth Year/Graduation Project/1st semester/Gap Analysis/v4.0/rently_2026/Gap_Analysis/notebooks/../src/results/monitoring_reports/snapshot_week_2_2026-01-09.csv


---

## Compute KPIs & Monitoring Metrics

In [5]:
kpis = compute_kpis(gap_summary)

print("Key Performance Indicators:")
for key, value in kpis.items():
    print(f"  {key}: {value}")

Key Performance Indicators:
  total_categories: 37
  avg_gap_score: 402.16
  median_gap_score: 0.5
  max_gap_score: 3776.75
  critical_categories: 16
  critical_pct: 43.2
  total_supply: 293
  total_demand: 84174



---

## Identify Alerts & Flags

In [6]:
analyzer = GapChangeAnalyzer()
gap_with_changes = analyzer.compute_gap_change(gap_summary)

flagged_categories = analyzer.identify_categories_needing_attention(gap_with_changes)

print("Categories Requiring Attention:")
display(flagged_categories[flagged_categories['priority'] != 'Normal'].head(15))

Categories Requiring Attention:


,category,gap_score,gap_change_pct,alerts,priority
0,Bottles and Cages,"3,776.75",0.00,Critical Alert,High
1,Tires and Tubes,"2,481.08",0.00,Critical Alert,High
2,Caps,"2,076.00",0.00,Critical Alert,High
3,Fenders,"1,980.50",0.00,Critical Alert,High
4,Helmets,"1,508.75",0.00,Critical Alert,High
5,Cleaners,853.50,0.00,Critical Alert,High
6,Gloves,377.86,0.00,Critical Alert,High
7,Hydration Packs,348.00,0.00,Critical Alert,High
8,Jerseys,346.00,0.00,Critical Alert,High
9,Socks,212.80,0.00,Critical Alert,High



---

## Generate Action Plans

In [7]:
planner = ActionPlanner()
action_plan_dict = planner.generate_category_action_plan(gap_with_changes, week_number=week_number)

print(f"Action plan generated for {len(action_plan_dict['action_plans'])} categories")
print(f"\nSample action plan for top 3 categories:\n")

for plan in action_plan_dict['action_plans'][:3]:
    print(f"Category: {plan['category']} | Gap Score: {plan['gap_score']:.2f}")
    print(f"  Operational: {plan['operational_actions'][0] if plan['operational_actions'] else 'None'}")
    print(f"  Strategic: {plan['strategic_actions'][0] if plan['strategic_actions'] else 'None'}\n")

Action plan generated for 37 categories

Sample action plan for top 3 categories:

Category: Bottles and Cages | Gap Score: 3776.75
  Operational: Launch supply recruitment campaign targeting lenders with Bottles and Cages items
  Strategic: Develop supplier partnership strategy for Bottles and Cages expansion

Category: Tires and Tubes | Gap Score: 2481.08
  Operational: Launch supply recruitment campaign targeting lenders with Tires and Tubes items
  Strategic: Develop supplier partnership strategy for Tires and Tubes expansion

Category: Caps | Gap Score: 2076.00
  Operational: Launch supply recruitment campaign targeting lenders with Caps items
  Strategic: Develop supplier partnership strategy for Caps expansion



---

## Export Action Plans

In [8]:
json_export_path = planner.export_action_plan_json(action_plan_dict, week_number=week_number)
text_export_path = planner.export_action_plan_text(action_plan_dict, week_number=week_number)

print(f"Action plans exported:")
print(f"  JSON: {json_export_path}")
print(f"  TXT: {text_export_path}")

with open(text_export_path, 'r') as f:
    print(f"\n{f.read()[:1000]}...")

Action plans exported:
  JSON: /home/hamza/All Data To Transfer/VIF/Materials/Fourth Year/Graduation Project/1st semester/Gap Analysis/v4.0/rently_2026/Gap_Analysis/notebooks/../results/monitoring_reports/action_plan_week_2.json
  TXT: /home/hamza/All Data To Transfer/VIF/Materials/Fourth Year/Graduation Project/1st semester/Gap Analysis/v4.0/rently_2026/Gap_Analysis/notebooks/../results/monitoring_reports/action_plan_week_2.txt

ACTION PLAN - WEEK 2 (2026-01-09)

Total Categories Monitored: 37

----------------------------------------------------------------------------------------------------
Category: Bottles and Cages
Gap Score: 3776.75 | Status: Critical Gap
Supply: 3 | Demand: 15,106
Gap Change: +0.0%

OPERATIONAL ACTIONS (Immediate):
  1. Launch supply recruitment campaign targeting lenders with Bottles and Cages items
  2. Offer 3-month listing incentive (10% commission reduction) for Bottles and Cages
  3. Create Bottles and Cages category highlight page with featured rewards



---

## Create Monitoring Visualizations

In [9]:
output_dir = Path("../results/monitoring_dashboards")
output_dir.mkdir(parents=True, exist_ok=True)

kpi_fig = create_monitoring_kpi_cards(kpis)
kpi_fig.show()

export_plotly_chart(kpi_fig, output_dir / "01_monitoring_kpis.html", format='html')
print("KPI cards exported")

KPI cards exported


In [10]:
alert_table_fig = create_alert_table(flagged_categories)
alert_table_fig.show()

export_plotly_chart(alert_table_fig, output_dir / "02_active_alerts.html", format='html')
print("Alert table exported")

Alert table exported


In [11]:
balance_fig = create_supply_demand_balance_chart(gap_summary)
balance_fig.show()

export_plotly_chart(balance_fig, output_dir / "03_supply_demand_balance.html", format='html')
print("Supply-demand balance chart exported")

Supply-demand balance chart exported


## Monitoring Dashboard Summary

In [12]:
print("=" * 100)
print("MONITORING & ACTION PLAN SUMMARY")
print("=" * 100)

print(f"\nMonitoring Period: Week {week_number} ({datetime.now().strftime('%Y-%m-%d')})")
print(f"\nKPI Summary:")
print(f"  Total Categories: {kpis['total_categories']}")
print(f"  Average Gap Score: {kpis['avg_gap_score']:.2f}")
print(f"  Critical Categories: {kpis['critical_categories']} ({kpis['critical_pct']}%)")
print(f"  Total Supply: {kpis['total_supply']:,}")
print(f"  Total Demand: {kpis['total_demand']:,}")

print(f"\nAlert Summary:")
high_priority = len(flagged_categories[flagged_categories['priority'] == 'High'])
medium_priority = len(flagged_categories[flagged_categories['priority'] == 'Medium'])
print(f"  High Priority: {high_priority}")
print(f"  Medium Priority: {medium_priority}")

print(f"\nTop 5 Categories Requiring Action:")
for idx, (_, row) in enumerate(gap_with_changes.nlargest(5, 'gap_score').iterrows(), 1):
    print(f"  {idx}. {row['category']} | Gap: {row['gap_score']:.2f} | Status: {row['gap_status']}")

print(f"\nExported Deliverables:")
print(f"  - Monitoring Snapshot (JSON/CSV)")
print(f"  - Action Plan (JSON/TXT)")
print(f"  - KPI Cards Visualization")
print(f"  - Alert Table Visualization")
print(f"  - Supply-Demand Balance Chart")

print("=" * 100)

MONITORING & ACTION PLAN SUMMARY

Monitoring Period: Week 2 (2026-01-09)

KPI Summary:
  Total Categories: 37
  Average Gap Score: 402.16
  Critical Categories: 16 (43.2%)
  Total Supply: 293
  Total Demand: 84,174

Alert Summary:
  High Priority: 17
  Medium Priority: 0

Top 5 Categories Requiring Action:
  1. Bottles and Cages | Gap: 3776.75 | Status: Critical Gap
  2. Tires and Tubes | Gap: 2481.08 | Status: Critical Gap
  3. Caps | Gap: 2076.00 | Status: Critical Gap
  4. Fenders | Gap: 1980.50 | Status: Critical Gap
  5. Helmets | Gap: 1508.75 | Status: Critical Gap

Exported Deliverables:
  - Monitoring Snapshot (JSON/CSV)
  - Action Plan (JSON/TXT)
  - KPI Cards Visualization
  - Alert Table Visualization
  - Supply-Demand Balance Chart


---